In [1]:
from bbq.decoder import d_osd, bp_osd, belief_propagation, _syn_inv_permute_field
# from bbq.utils import det_to_err, err_to_det, find_pivots, rref
import numpy as np
# from ldpc import BpDecoder, bposd_decoder
from bbq.polynomial import Polynomial
from bbq.bbq_code import BivariateBicycle
# import matplotlib.pyplot as plt
# import json
# import datetime
# import galois

In [2]:
def simulate(field, h, l, max_iter, num_failures, physical_error):

    results = []

    for p in physical_error:

        failures = 0
        num_trials = 0

        while failures < num_failures:
            # Generate syndrome
            n_qudits = h.shape[1]
            error = np.zeros(n_qudits, dtype=int)
            error_mask = np.random.rand(n_qudits) < p
            for i in np.where(error_mask)[0]:
                error[i] = np.random.randint(1, field)
            syndrome = (h @ error) % field

            # Construct error probability
            channel_prob_x = np.ones(n_qudits) * p

            x_prior = np.zeros((len(channel_prob_x), field), dtype=float)

            for i, prob in enumerate(channel_prob_x):
                x_prior[i, 0] = 1 - prob
                for j in range(1, field):
                    x_prior[i, j] = prob / (field - 1)

            # Decode
            # guessed_error, decoder_success, bp_success, posterior = belief_propagation(field, h, syndrome, x_prior, max_iter, debug=True)
            guessed_error, decoder_success, bp_success, posterior = bp_osd(field, h, syndrome, x_prior, max_iter, order=0, debug=True)
            error_difference = (error - guessed_error) % field
            logical_effect = (np.array(l) @ error_difference) % field

            # Check success
            # if np.any(logical_effect != 0) or not decoder_success:
            if np.any(logical_effect != 0):
                failures += 1
                print(f'Found {failures} / {num_failures}, with num_trials : {num_trials}')
            elif num_trials % 100 == 0:
                print(f'UPDATE: Found {failures} / {num_failures}, with num_trials : {num_trials}')

            num_trials += 1

        results.append(num_trials)

        print(f'Finished p={p} with num_trials={num_trials}')
        print('------------------------------------------------------------------')
    return results

In [3]:
a = Polynomial(2, np.array([[1, 0], [1, 0]]))
b = Polynomial(2, np.array([[1, 1], [0, 0]]))
bb2 = BivariateBicycle(a, b, 3, 3, 1)
h2 = bb2.hx
l2 = bb2.x_logicals
bb25 = BivariateBicycle(a, b, 5, 5, 1)
h25 = bb25.hx
l25 = bb25.x_logicals
bb27 = BivariateBicycle(a, b, 7, 7, 1)
h27 = bb27.hx
l27 = bb27.x_logicals

a = Polynomial(3, np.array([[1, 0], [-1, 0]]))
b = Polynomial(3, np.array([[1, -1], [0, 0]]))
bb3 = BivariateBicycle(a, b, 3, 3, 1)
h3 = bb3.hx
l3 = bb3.x_logicals
bb35 = BivariateBicycle(a, b, 5, 5, 1)
h35 = bb35.hx
l35 = bb35.x_logicals
bb37 = BivariateBicycle(a, b, 7, 7, 1)
h37 = bb37.hx
l37 = bb37.x_logicals

a = Polynomial(5, np.array([[1, 0], [-1, 0]]))
b = Polynomial(5, np.array([[1, -1], [0, 0]]))
bb5 = BivariateBicycle(a, b, 3, 3, 1)
h5 = bb5.hx
l5 = bb5.x_logicals
bb55 = BivariateBicycle(a, b, 5, 5, 1)
h55 = bb55.hx
l55 = bb55.x_logicals
bb57 = BivariateBicycle(a, b, 7, 7, 1)
h57 = bb57.hx
l57 = bb57.x_logicals

max_iter = 1000
num_failures = 3
long_physical_error = np.logspace(-2.7, -1, 10)
physical_error = np.flip(np.logspace(-1.7, -0.7, 10))
ext_physical_error = np.flip(np.logspace(-1, -0.8, 10))

In [4]:
results = np.flip(simulate(5, h5, l5, max_iter, 50, ext_physical_error))

C:\Users\niall\Documents\repos\bbqudit\bbq\decoder.py:105: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, error, :] = sub_convolution[syn_inv_permutation[errs[j, 1], :]]


Found 1 / 50, with num_trials : 0
Found 2 / 50, with num_trials : 1
Found 3 / 50, with num_trials : 6
Found 4 / 50, with num_trials : 8
Found 5 / 50, with num_trials : 11
Found 6 / 50, with num_trials : 13
Found 7 / 50, with num_trials : 14
Found 8 / 50, with num_trials : 17
Found 9 / 50, with num_trials : 24
Found 10 / 50, with num_trials : 25
Found 11 / 50, with num_trials : 28
Found 12 / 50, with num_trials : 34
Found 13 / 50, with num_trials : 37
Found 14 / 50, with num_trials : 39
Found 15 / 50, with num_trials : 40
Found 16 / 50, with num_trials : 43
Found 17 / 50, with num_trials : 44
Found 18 / 50, with num_trials : 46
Found 19 / 50, with num_trials : 47
Found 20 / 50, with num_trials : 48
Found 21 / 50, with num_trials : 54
Found 22 / 50, with num_trials : 56
Found 23 / 50, with num_trials : 62
Found 24 / 50, with num_trials : 63
Found 25 / 50, with num_trials : 75
Found 26 / 50, with num_trials : 77
Found 27 / 50, with num_trials : 78
Found 28 / 50, with num_trials : 80
Found

In [5]:
results

array([342, 287, 288, 219, 260, 193, 174, 181, 156, 157])

In [12]:
results = np.flip(simulate(5, h55, l55, max_iter, 50, ext_physical_error))

UPDATE: Found 0 / 50, with num_trials : 0


C:\Users\niall\Documents\repos\bbqudit\bbq\decoder.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, error, :] = sub_convolution[syn_inv_permutation[errs[j, 1], :]]


Found 1 / 50, with num_trials : 3
Found 2 / 50, with num_trials : 10
Found 3 / 50, with num_trials : 21
Found 4 / 50, with num_trials : 25
Found 5 / 50, with num_trials : 26
Found 6 / 50, with num_trials : 28
Found 7 / 50, with num_trials : 29
Found 8 / 50, with num_trials : 32
Found 9 / 50, with num_trials : 34
Found 10 / 50, with num_trials : 39
Found 11 / 50, with num_trials : 42
Found 12 / 50, with num_trials : 44
Found 13 / 50, with num_trials : 49
Found 14 / 50, with num_trials : 56
Found 15 / 50, with num_trials : 61
Found 16 / 50, with num_trials : 66
Found 17 / 50, with num_trials : 68
Found 18 / 50, with num_trials : 70
Found 19 / 50, with num_trials : 73
Found 20 / 50, with num_trials : 83
Found 21 / 50, with num_trials : 84
Found 22 / 50, with num_trials : 87
Found 23 / 50, with num_trials : 90
Found 24 / 50, with num_trials : 92
UPDATE: Found 24 / 50, with num_trials : 100
Found 25 / 50, with num_trials : 103
Found 26 / 50, with num_trials : 112
Found 27 / 50, with num_tri

In [13]:
results

array([756, 801, 488, 513, 506, 356, 325, 201, 205, 194])

In [14]:
results = np.flip(simulate(5, h57, l57, max_iter, 50, ext_physical_error))

UPDATE: Found 0 / 50, with num_trials : 0
Found 1 / 50, with num_trials : 1
Found 2 / 50, with num_trials : 13
Found 3 / 50, with num_trials : 17
Found 4 / 50, with num_trials : 20
Found 5 / 50, with num_trials : 37
Found 6 / 50, with num_trials : 48
Found 7 / 50, with num_trials : 50
Found 8 / 50, with num_trials : 51
Found 9 / 50, with num_trials : 53
Found 10 / 50, with num_trials : 64
Found 11 / 50, with num_trials : 72
Found 12 / 50, with num_trials : 75
Found 13 / 50, with num_trials : 82
Found 14 / 50, with num_trials : 94
Found 15 / 50, with num_trials : 95
Found 16 / 50, with num_trials : 100
Found 17 / 50, with num_trials : 103
Found 18 / 50, with num_trials : 104
Found 19 / 50, with num_trials : 105
Found 20 / 50, with num_trials : 117
Found 21 / 50, with num_trials : 122
Found 22 / 50, with num_trials : 125
Found 23 / 50, with num_trials : 129
Found 24 / 50, with num_trials : 133
Found 25 / 50, with num_trials : 135
Found 26 / 50, with num_trials : 137
Found 27 / 50, with n

In [15]:
results

array([1271, 1414, 1036,  987,  778,  489,  470,  324,  232,  224])

In [6]:
results = np.flip(simulate(5, h5, l5, max_iter, 50, ext_physical_error))

C:\Users\niall\Documents\repos\bbqudit\bbq\decoder.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, error, :] = sub_convolution[syn_inv_permutation[errs[j, 1], :]]


UPDATE: Found 0 / 50, with num_trials : 0
Found 1 / 50, with num_trials : 3
Found 2 / 50, with num_trials : 7
Found 3 / 50, with num_trials : 10
Found 4 / 50, with num_trials : 12
Found 5 / 50, with num_trials : 15
Found 6 / 50, with num_trials : 16
Found 7 / 50, with num_trials : 17
Found 8 / 50, with num_trials : 19
Found 9 / 50, with num_trials : 25
Found 10 / 50, with num_trials : 27
Found 11 / 50, with num_trials : 28
Found 12 / 50, with num_trials : 29
Found 13 / 50, with num_trials : 32
Found 14 / 50, with num_trials : 36
Found 15 / 50, with num_trials : 38
Found 16 / 50, with num_trials : 41
Found 17 / 50, with num_trials : 42
Found 18 / 50, with num_trials : 45
Found 19 / 50, with num_trials : 47
Found 20 / 50, with num_trials : 48
Found 21 / 50, with num_trials : 64
Found 22 / 50, with num_trials : 66
Found 23 / 50, with num_trials : 70
Found 24 / 50, with num_trials : 71
Found 25 / 50, with num_trials : 73
Found 26 / 50, with num_trials : 76
Found 27 / 50, with num_trials : 

In [7]:
results

array([338, 299, 243, 332, 177, 191, 167, 171, 132, 135])

In [8]:
results = np.flip(simulate(5, h55, l55, max_iter, 50, ext_physical_error))

UPDATE: Found 0 / 50, with num_trials : 0
Found 1 / 50, with num_trials : 1
Found 2 / 50, with num_trials : 12
Found 3 / 50, with num_trials : 17
Found 4 / 50, with num_trials : 29
Found 5 / 50, with num_trials : 33
Found 6 / 50, with num_trials : 45
Found 7 / 50, with num_trials : 51
Found 8 / 50, with num_trials : 54
Found 9 / 50, with num_trials : 56
Found 10 / 50, with num_trials : 57
Found 11 / 50, with num_trials : 61
Found 12 / 50, with num_trials : 62
Found 13 / 50, with num_trials : 65
Found 14 / 50, with num_trials : 66
Found 15 / 50, with num_trials : 70
Found 16 / 50, with num_trials : 71
Found 17 / 50, with num_trials : 74
Found 18 / 50, with num_trials : 76
Found 19 / 50, with num_trials : 80
Found 20 / 50, with num_trials : 87
Found 21 / 50, with num_trials : 94
Found 22 / 50, with num_trials : 96
Found 23 / 50, with num_trials : 97
Found 24 / 50, with num_trials : 98
UPDATE: Found 24 / 50, with num_trials : 100
Found 25 / 50, with num_trials : 103
Found 26 / 50, with nu

In [9]:
results

array([731, 589, 629, 415, 374, 361, 279, 305, 229, 202])

In [10]:
results = np.flip(simulate(5, h57, l57, max_iter, 50, ext_physical_error))

UPDATE: Found 0 / 50, with num_trials : 0
Found 1 / 50, with num_trials : 4
Found 2 / 50, with num_trials : 7
Found 3 / 50, with num_trials : 9
Found 4 / 50, with num_trials : 14
Found 5 / 50, with num_trials : 21
Found 6 / 50, with num_trials : 28
Found 7 / 50, with num_trials : 32
Found 8 / 50, with num_trials : 33
Found 9 / 50, with num_trials : 35
Found 10 / 50, with num_trials : 36
Found 11 / 50, with num_trials : 38
Found 12 / 50, with num_trials : 45
Found 13 / 50, with num_trials : 48
Found 14 / 50, with num_trials : 50
Found 15 / 50, with num_trials : 51
Found 16 / 50, with num_trials : 53
Found 17 / 50, with num_trials : 57
Found 18 / 50, with num_trials : 58
Found 19 / 50, with num_trials : 59
Found 20 / 50, with num_trials : 61
Found 21 / 50, with num_trials : 68
Found 22 / 50, with num_trials : 73
Found 23 / 50, with num_trials : 82
Found 24 / 50, with num_trials : 90
Found 25 / 50, with num_trials : 94
Found 26 / 50, with num_trials : 95
Found 27 / 50, with num_trials : 9

In [11]:
results

array([1707, 1538, 1017,  757,  662,  503,  364,  337,  285,  204])

In [ ]:
results = np.flip(simulate(5, h57, l57, max_iter, 100, ext_physical_error))
results

UPDATE: Found 0 / 100, with num_trials : 0


C:\Users\niall\Documents\repos\bbqudit\bbq\decoder.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, error, :] = sub_convolution[syn_inv_permutation[errs[j, 1], :]]


Found 1 / 100, with num_trials : 1
Found 2 / 100, with num_trials : 2
Found 3 / 100, with num_trials : 5
Found 4 / 100, with num_trials : 15
Found 5 / 100, with num_trials : 16
Found 6 / 100, with num_trials : 17
Found 7 / 100, with num_trials : 20
Found 8 / 100, with num_trials : 23
Found 9 / 100, with num_trials : 28
Found 10 / 100, with num_trials : 30
Found 11 / 100, with num_trials : 31
Found 12 / 100, with num_trials : 32
Found 13 / 100, with num_trials : 33
Found 14 / 100, with num_trials : 34
Found 15 / 100, with num_trials : 38
Found 16 / 100, with num_trials : 43
Found 17 / 100, with num_trials : 59
Found 18 / 100, with num_trials : 63
Found 19 / 100, with num_trials : 66
Found 20 / 100, with num_trials : 67
Found 21 / 100, with num_trials : 68
Found 22 / 100, with num_trials : 72
Found 23 / 100, with num_trials : 74
Found 24 / 100, with num_trials : 82
Found 25 / 100, with num_trials : 84
Found 26 / 100, with num_trials : 87
Found 27 / 100, with num_trials : 89
Found 28 / 10

In [16]:
results = np.flip(simulate(2, h2, l2, max_iter, 100, ext_physical_error))
results

Found 1 / 100, with num_trials : 0
Found 2 / 100, with num_trials : 4
Found 3 / 100, with num_trials : 5
Found 4 / 100, with num_trials : 7
Found 5 / 100, with num_trials : 8
Found 6 / 100, with num_trials : 9
Found 7 / 100, with num_trials : 10
Found 8 / 100, with num_trials : 11
Found 9 / 100, with num_trials : 15
Found 10 / 100, with num_trials : 16
Found 11 / 100, with num_trials : 17
Found 12 / 100, with num_trials : 19
Found 13 / 100, with num_trials : 21
Found 14 / 100, with num_trials : 22
Found 15 / 100, with num_trials : 25
Found 16 / 100, with num_trials : 28
Found 17 / 100, with num_trials : 31
Found 18 / 100, with num_trials : 32
Found 19 / 100, with num_trials : 35
Found 20 / 100, with num_trials : 41
Found 21 / 100, with num_trials : 42
Found 22 / 100, with num_trials : 43
Found 23 / 100, with num_trials : 48
Found 24 / 100, with num_trials : 53
Found 25 / 100, with num_trials : 54
Found 26 / 100, with num_trials : 56
Found 27 / 100, with num_trials : 58
Found 28 / 100, 

array([441, 406, 359, 316, 329, 286, 296, 278, 227, 235])

In [17]:
results = np.flip(simulate(2, h25, l25, max_iter, 100, ext_physical_error))
results

UPDATE: Found 0 / 100, with num_trials : 0
Found 1 / 100, with num_trials : 1
Found 2 / 100, with num_trials : 2
Found 3 / 100, with num_trials : 3
Found 4 / 100, with num_trials : 5
Found 5 / 100, with num_trials : 6
Found 6 / 100, with num_trials : 7
Found 7 / 100, with num_trials : 8
Found 8 / 100, with num_trials : 10
Found 9 / 100, with num_trials : 12
Found 10 / 100, with num_trials : 13
Found 11 / 100, with num_trials : 14
Found 12 / 100, with num_trials : 18
Found 13 / 100, with num_trials : 20
Found 14 / 100, with num_trials : 23
Found 15 / 100, with num_trials : 24
Found 16 / 100, with num_trials : 25
Found 17 / 100, with num_trials : 26
Found 18 / 100, with num_trials : 27
Found 19 / 100, with num_trials : 28
Found 20 / 100, with num_trials : 29
Found 21 / 100, with num_trials : 32
Found 22 / 100, with num_trials : 33
Found 23 / 100, with num_trials : 36
Found 24 / 100, with num_trials : 39
Found 25 / 100, with num_trials : 41
Found 26 / 100, with num_trials : 42
Found 27 / 

array([418, 372, 357, 283, 276, 265, 259, 243, 207, 185])

In [18]:
results = np.flip(simulate(2, h27, l27, max_iter, 100, ext_physical_error))
results

Found 1 / 100, with num_trials : 0
Found 2 / 100, with num_trials : 1
Found 3 / 100, with num_trials : 2
Found 4 / 100, with num_trials : 5
Found 5 / 100, with num_trials : 7
Found 6 / 100, with num_trials : 8
Found 7 / 100, with num_trials : 9
Found 8 / 100, with num_trials : 10
Found 9 / 100, with num_trials : 12
Found 10 / 100, with num_trials : 13
Found 11 / 100, with num_trials : 14
Found 12 / 100, with num_trials : 18
Found 13 / 100, with num_trials : 20
Found 14 / 100, with num_trials : 21
Found 15 / 100, with num_trials : 24
Found 16 / 100, with num_trials : 26
Found 17 / 100, with num_trials : 27
Found 18 / 100, with num_trials : 29
Found 19 / 100, with num_trials : 30
Found 20 / 100, with num_trials : 31
Found 21 / 100, with num_trials : 32
Found 22 / 100, with num_trials : 33
Found 23 / 100, with num_trials : 35
Found 24 / 100, with num_trials : 36
Found 25 / 100, with num_trials : 38
Found 26 / 100, with num_trials : 40
Found 27 / 100, with num_trials : 41
Found 28 / 100, w

array([457, 440, 337, 300, 251, 263, 208, 185, 178, 156])

In [19]:
results = np.flip(simulate(3, h3, l3, max_iter, 100, ext_physical_error))
results

UPDATE: Found 0 / 100, with num_trials : 0
Found 1 / 100, with num_trials : 3
Found 2 / 100, with num_trials : 13
Found 3 / 100, with num_trials : 17
Found 4 / 100, with num_trials : 21
Found 5 / 100, with num_trials : 22
Found 6 / 100, with num_trials : 24
Found 7 / 100, with num_trials : 30
Found 8 / 100, with num_trials : 31
Found 9 / 100, with num_trials : 32
Found 10 / 100, with num_trials : 33
Found 11 / 100, with num_trials : 34
Found 12 / 100, with num_trials : 35
Found 13 / 100, with num_trials : 40
Found 14 / 100, with num_trials : 41
Found 15 / 100, with num_trials : 49
Found 16 / 100, with num_trials : 50
Found 17 / 100, with num_trials : 51
Found 18 / 100, with num_trials : 54
Found 19 / 100, with num_trials : 56
Found 20 / 100, with num_trials : 58
Found 21 / 100, with num_trials : 59
Found 22 / 100, with num_trials : 61
Found 23 / 100, with num_trials : 69
Found 24 / 100, with num_trials : 71
Found 25 / 100, with num_trials : 73
Found 26 / 100, with num_trials : 74
Found

array([556, 516, 430, 455, 389, 343, 292, 332, 280, 268])

In [20]:
results = np.flip(simulate(3, h35, l35, max_iter, 100, ext_physical_error))
results

Found 1 / 100, with num_trials : 0
Found 2 / 100, with num_trials : 2
Found 3 / 100, with num_trials : 3
Found 4 / 100, with num_trials : 4
Found 5 / 100, with num_trials : 13
Found 6 / 100, with num_trials : 15
Found 7 / 100, with num_trials : 18
Found 8 / 100, with num_trials : 22
Found 9 / 100, with num_trials : 24
Found 10 / 100, with num_trials : 25
Found 11 / 100, with num_trials : 30
Found 12 / 100, with num_trials : 31
Found 13 / 100, with num_trials : 32
Found 14 / 100, with num_trials : 33
Found 15 / 100, with num_trials : 34
Found 16 / 100, with num_trials : 35
Found 17 / 100, with num_trials : 36
Found 18 / 100, with num_trials : 37
Found 19 / 100, with num_trials : 38
Found 20 / 100, with num_trials : 45
Found 21 / 100, with num_trials : 47
Found 22 / 100, with num_trials : 49
Found 23 / 100, with num_trials : 51
Found 24 / 100, with num_trials : 52
Found 25 / 100, with num_trials : 59
Found 26 / 100, with num_trials : 60
Found 27 / 100, with num_trials : 61
Found 28 / 100

array([852, 613, 604, 511, 395, 367, 317, 301, 251, 250])

In [21]:
results = np.flip(simulate(3, h37, l37, max_iter, 100, ext_physical_error))
results

Found 1 / 100, with num_trials : 0
Found 2 / 100, with num_trials : 2
Found 3 / 100, with num_trials : 3
Found 4 / 100, with num_trials : 4
Found 5 / 100, with num_trials : 10
Found 6 / 100, with num_trials : 12
Found 7 / 100, with num_trials : 14
Found 8 / 100, with num_trials : 15
Found 9 / 100, with num_trials : 16
Found 10 / 100, with num_trials : 17
Found 11 / 100, with num_trials : 18
Found 12 / 100, with num_trials : 21
Found 13 / 100, with num_trials : 22
Found 14 / 100, with num_trials : 23
Found 15 / 100, with num_trials : 25
Found 16 / 100, with num_trials : 29
Found 17 / 100, with num_trials : 30
Found 18 / 100, with num_trials : 33
Found 19 / 100, with num_trials : 36
Found 20 / 100, with num_trials : 37
Found 21 / 100, with num_trials : 39
Found 22 / 100, with num_trials : 42
Found 23 / 100, with num_trials : 43
Found 24 / 100, with num_trials : 44
Found 25 / 100, with num_trials : 45
Found 26 / 100, with num_trials : 49
Found 27 / 100, with num_trials : 50
Found 28 / 100

array([1156,  937,  668,  630,  407,  458,  349,  312,  212,  218])

In [4]:
new_ext_physical_error = np.flip(np.logspace(-0.8, -0.64, 10)[1:7])

In [6]:
np.flip(simulate(5, h57, l57, max_iter, 100, new_ext_physical_error))

Found 1 / 100, with num_trials : 0
Found 2 / 100, with num_trials : 1
Found 3 / 100, with num_trials : 2
Found 4 / 100, with num_trials : 6
Found 5 / 100, with num_trials : 7
Found 6 / 100, with num_trials : 9
Found 7 / 100, with num_trials : 10
Found 8 / 100, with num_trials : 11
Found 9 / 100, with num_trials : 14
Found 10 / 100, with num_trials : 16
Found 11 / 100, with num_trials : 18
Found 12 / 100, with num_trials : 19
Found 13 / 100, with num_trials : 21
Found 14 / 100, with num_trials : 27
Found 15 / 100, with num_trials : 29
Found 16 / 100, with num_trials : 31
Found 17 / 100, with num_trials : 32
Found 18 / 100, with num_trials : 33
Found 19 / 100, with num_trials : 34
Found 20 / 100, with num_trials : 35
Found 21 / 100, with num_trials : 36
Found 22 / 100, with num_trials : 38
Found 23 / 100, with num_trials : 41
Found 24 / 100, with num_trials : 42
Found 25 / 100, with num_trials : 43
Found 26 / 100, with num_trials : 46
Found 27 / 100, with num_trials : 48
Found 28 / 100, 

array([389, 332, 265, 236, 223, 197])

In [5]:
np.flip(simulate(5, h57, l57, max_iter, 100, new_ext_physical_error))

C:\Users\niall\Documents\repos\bbqudit\bbq\decoder.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  P[i, error, :] = sub_convolution[syn_inv_permutation[errs[j, 1], :]]


Found 1 / 100, with num_trials : 0
Found 2 / 100, with num_trials : 1
Found 3 / 100, with num_trials : 2
Found 4 / 100, with num_trials : 4
Found 5 / 100, with num_trials : 6
Found 6 / 100, with num_trials : 7
Found 7 / 100, with num_trials : 10
Found 8 / 100, with num_trials : 15
Found 9 / 100, with num_trials : 20
Found 10 / 100, with num_trials : 21
Found 11 / 100, with num_trials : 24
Found 12 / 100, with num_trials : 28
Found 13 / 100, with num_trials : 29
Found 14 / 100, with num_trials : 33
Found 15 / 100, with num_trials : 34
Found 16 / 100, with num_trials : 35
Found 17 / 100, with num_trials : 36
Found 18 / 100, with num_trials : 37
Found 19 / 100, with num_trials : 38
Found 20 / 100, with num_trials : 41
Found 21 / 100, with num_trials : 44
Found 22 / 100, with num_trials : 47
Found 23 / 100, with num_trials : 48
Found 24 / 100, with num_trials : 50
Found 25 / 100, with num_trials : 52
Found 26 / 100, with num_trials : 56
Found 27 / 100, with num_trials : 57
Found 28 / 100, 

array([425, 289, 296, 235, 212, 197])